# Tiles with Borders

We're exploring tiles of size $2^n$, where $n \geqslant 3$, since tiles of size with the following:





In [4]:
2^0;

(%o4)                                  1

In [5]:
2^1;

(%o5)                                  2

would result in a collection of tiles too small in size to be useful.

We'll start with tiles of size where $n = 3$, giving us one tile, four tiles, and 16 tiles like the following:

[![Example 4x4 tiles](files/images/4x4-tile-examples-thumb.png)](files/images/4x4-tile-examples.png)

The example above shows tiles with borders and where those borders may have interaction with other tiles. In particular:

* <font color="#C27BA0" size="10px">&#9632;</font> Cells with this colour are conflcit-free, contention-free zone: they will never have shared data for time steps  $t + 1$.
* <font color="#6D9EEB" size="10px">&#9632;</font> Cells with this colour are those part of a tile's outer border. Since they are part of the outer border, with no overlap with cells of other tiles, they can only be considered temporarily conflict-free. This is due to the fact that the tiles could be moved or other tiles could be placed against their borders. As such, at every time step, we should check if a new tile has been placed adjacent; if not, we may continue to regard it as conflict-free for time steps  $t + 1$.
* <font color="#3C78D8" size="10px">&#9632;</font> Dark blue cells are outer border cells of a tile that are adjacent to other tiles's outer border cells and could have shared data between two cells of two different tiles for time steps $t + 1$.
* <font color="#8E7CC3" size="10px">&#9632;</font> Light purple cells are non-outer border cells of two adjacent tiless' non-outer border cells and could have shared data between two cells of two different tiles for time steps $t + 1$.
* <font color="#674EA7" size="10px">&#9632;</font> Dark purple cells are non-outer border cells of four adjacent tiles' non-outer border cells and could have shared data between fours cells of four different tiles for time steps $t + 1$.

The case of $n = 2$ isn't all that interesting from a practical perspective, but we'll look at it for the purposes of mathematical consistency:

[![Example 2x2 tiles](files/images/2x2-tile-examples-thumb.png)](files/images/2x2-tile-examples.png)

For $n = 4$, we have this set of tiles (one tile, four tiles, and 16 tiles):

[![Example 8x8 tiles](files/images/8x8-tile-examples-thumb.png)](files/images/8x8-tile-examples.png)
